In [380]:
import pandas as pd 
import numpy as np 
import requests
import matplotlib as plt 
import seaborn as sns 
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [328]:
url = "http://www.isthe.com/chongo/tech/astro/HR-temp-mass-table-byhrclass.html#below"

In [349]:
star_data = pd.DataFrame()
html = pd.read_html(url)
for i in html:
    star_data = pd.concat([star_data, i])

In [350]:
len(star_data)

1242

In [351]:
star_data.drop([0, 1, 2], axis=1, inplace=True)

In [352]:
star_data.dropna(axis=0, inplace=True)

In [353]:
len(star_data)

1100

In [354]:
star_data['color_decimal'] = ""

In [355]:
def rgb_cleaner(rgb_text):
    temp_split = rgb_text.split()
    red = int(temp_split[0])
    green = int(temp_split[1])
    blue = int(temp_split[2])
    color_hex = '%02x%02x%02x' % (red, green, blue)
    color_finished = int(color_hex, 16)
    return color_finished


In [356]:
star_data['color_decimal'] = [rgb_cleaner(i) for i in star_data['Star ColorRGB 0-255']]

In [358]:
star_data.drop(['Star ColorRGB 0-255'], axis=1, inplace=True)

In [359]:
star_data['star_family'] = [i[0] for i in star_data['StellarType']]

In [319]:
# star_data = pd.get_dummies(star_data, columns=['star_family'])

In [360]:
star_data.tail()

,Abs MagMv,Bolo CorrBC(Temp),Bolo MagMbol,Color IndexB-V,LuminosityLstar/Lsun,MassMstar/Msun,RadiusRstar/Rsun,StellarType,TempK,color_decimal,star_family
1231,13.1,-0.45,12.65,-0.07,0.000693,0.5,0.00890,DZ5,10080.0,13622015,D
1232,13.7,-0.20,13.50,0.09,0.000315,0.4,0.00864,DZ6,8400.0,14739199,D
1233,14.2,-0.09,14.11,0.34,0.000180,0.3,0.00889,DZ7,7200.0,15987711,D
1234,14.8,-0.10,14.70,0.55,0.000105,0.2,0.00887,DZ8,6300.0,16775157,D
1235,15.4,-0.22,15.18,0.74,0.000067,0.1,0.00898,DZ9,5600.0,16773089,D


In [361]:
star_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 1 to 1235
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Abs MagMv             1100 non-null   float64
 1   Bolo CorrBC(Temp)     1100 non-null   float64
 2   Bolo MagMbol          1100 non-null   float64
 3   Color IndexB-V        1100 non-null   float64
 4   LuminosityLstar/Lsun  1100 non-null   float64
 5   MassMstar/Msun        1100 non-null   float64
 6   RadiusRstar/Rsun      1100 non-null   float64
 7   StellarType           1100 non-null   object 
 8   TempK                 1100 non-null   float64
 9   color_decimal         1100 non-null   int64  
 10  star_family           1100 non-null   object 
dtypes: float64(8), int64(1), object(2)
memory usage: 103.1+ KB


In [340]:
import pandas_profiling

In [25]:
profile = star_data.profile_report(title='EDA of star dataset using Pandas Profiling')
# save the minified report
profile.to_file(output_file="star.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 59.52it/s]


In [362]:
# Set up x and y variables
X = star_data[['Abs MagMv', 'Bolo CorrBC(Temp)', 'Bolo MagMbol', 'Color IndexB-V', 'LuminosityLstar/Lsun', 'MassMstar/Msun', 'RadiusRstar/Rsun', 'TempK', 'color_decimal']]
y = star_data['star_family']

In [363]:
# Make training and testing set (testing is 20% of the data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=14)

## Logistic Regression

In [364]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [365]:
yhat_train = lr.predict(X_train)
y_pred = lr.predict(X_test)

In [366]:
y.nunique()

13

In [367]:
print('Accuracy = {:.5f}'.format(accuracy_score(y_test, y_pred)))
print('Precision = {:.5f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Recall = {:.5f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('F1 score = {:.5f}'.format(f1_score(y_test, y_pred, average='weighted')))

Accuracy = 0.22727
Precision = 0.12487
Recall = 0.22727
F1 score = 0.13313


In [293]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           A       0.00      0.00      0.00        13
           B       0.00      0.00      0.00        19
           C       0.00      0.00      0.00        18
           D       0.00      0.00      0.00         8
           F       0.00      0.00      0.00        20
           G       0.10      0.86      0.18        14
           K       0.00      0.00      0.00        16
           M       0.00      0.00      0.00        14
           N       0.50      0.08      0.13        13
           O       0.00      0.00      0.00        16
           R       0.27      0.25      0.26        16
           S       0.00      0.00      0.00        15
           W       0.40      0.87      0.55        38

    accuracy                           0.23       220
   macro avg       0.10      0.16      0.09       220
weighted avg       0.12      0.23      0.13       220



In [254]:
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score, plot_roc_curve, plot_precision_recall_curve, plot_confusion_matrix


## K Nearest Neighbors

In [368]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [369]:
knn_yhat_train = knn.predict(X_train)
knn_y_pred = knn.predict(X_test)

In [374]:
print(classification_report(y_test, knn_y_pred))

              precision    recall  f1-score   support

           A       0.67      0.77      0.71        13
           B       0.87      0.68      0.76        19
           C       0.14      0.22      0.17        18
           D       0.73      1.00      0.84         8
           F       0.89      0.85      0.87        20
           G       0.55      0.79      0.65        14
           K       0.44      0.44      0.44        16
           M       0.17      0.29      0.22        14
           N       0.00      0.00      0.00        13
           O       0.00      0.00      0.00        16
           R       0.12      0.06      0.08        16
           S       0.00      0.00      0.00        15
           W       0.61      0.71      0.66        38

    accuracy                           0.46       220
   macro avg       0.40      0.45      0.42       220
weighted avg       0.43      0.46      0.44       220



In [370]:
print('Accuracy = {:.5f}'.format(accuracy_score(y_test, knn_y_pred)))
print('Precision = {:.5f}'.format(precision_score(y_test, knn_y_pred, average='weighted')))
print('Recall = {:.5f}'.format(recall_score(y_test, knn_y_pred, average='weighted')))
print('F1 score = {:.5f}'.format(f1_score(y_test, knn_y_pred, average='weighted')))

Accuracy = 0.46364
Precision = 0.42628
Recall = 0.46364
F1 score = 0.43861


## Decision Tree

In [371]:
dt = DecisionTreeClassifier(min_samples_leaf=3)
dt.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [372]:
dt_yhat_train = dt.predict(X_train)
dt_y_pred = dt.predict(X_test)

In [373]:
print('Accuracy = {:.5f}'.format(accuracy_score(y_test, dt_y_pred)))
print('Precision = {:.5f}'.format(precision_score(y_test, dt_y_pred, average='weighted')))
print('Recall = {:.5f}'.format(recall_score(y_test, dt_y_pred, average='weighted')))
print('F1 score = {:.5f}'.format(f1_score(y_test, dt_y_pred, average='weighted')))

Accuracy = 0.51364
Precision = 0.52591
Recall = 0.51364
F1 score = 0.51709


## Random Forest

In [382]:
rf = RandomForestClassifier()
rf.fit(X_test, y_test)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [383]:
rf_yhat_train = rf.predict(X_train)
rf_y_pred = rf.predict(X_test)

In [384]:
print('Accuracy = {:.5f}'.format(accuracy_score(y_test, rf_y_pred)))
print('Precision = {:.5f}'.format(precision_score(y_test, rf_y_pred, average='weighted')))
print('Recall = {:.5f}'.format(recall_score(y_test, rf_y_pred, average='weighted')))
print('F1 score = {:.5f}'.format(f1_score(y_test, rf_y_pred, average='weighted')))

Accuracy = 0.91818
Precision = 0.91892
Recall = 0.91818
F1 score = 0.91588


TODO:   
-Logistic regression   
-Support vector machine  
-Naive Bayes

Split the first letter of each star off, put in new column. I will just predict that part since there are 1100 unique rows for the stars.
